<a href="https://www.kaggle.com/code/deepakchandhru/team-rizz-dl-2-task-1?scriptVersionId=195806667" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
import csv

df = pd.read_csv("/kaggle/input/medium-articles/medium_articles.csv")
print(df.head())

In [ ]:
df = df[['title','text']]
df.shape

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
df['text'] = df['text'].str.replace('\n','<|n|>')

In [ ]:
sample_blog = df['text'][10]
print(sample_blog)

In [ ]:
sentences = sent_tokenize(sample_blog)
sentences_clean = [re.sub(r'[^\w\s]', '', sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)
sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
max_len = max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant') for embedding in sentence_embeddings]


In [ ]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i, row_embedding in enumerate(sentence_embeddings):
    for j, column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph, max_iter=600)
top_sentence = {sentence: scores[index] for index, sentence in enumerate(sentences)}
sentNeeded = round(0.25 * len(sentences)) - 1
top_dict = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
summary = ""
for sent in sentences:
    if sent in top_dict.keys():
        summary += sent
print(summary)


In [ ]:
count = 0

def generateSummary(blog):
    global count
    count+=1
    print("Summarising Blog",count)
    
    try:
        sentences = sent_tokenize(sample_blog)
        sentences_clean = [re.sub(r'[^\w\s]', '', sentence.lower()) for sentence in sentences]
        stop_words = stopwords.words('english')
        sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
        w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)
        sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
        max_len = max([len(tokens) for tokens in sentence_tokens])
        sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant') for embedding in sentence_embeddings]
        similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
        for i, row_embedding in enumerate(sentence_embeddings):
            for j, column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph, max_iter=600)
        top_sentence = {sentence: scores[index] for index, sentence in enumerate(sentences)}
        sentNeeded = round(0.25 * len(sentences)) - 1
        top_dict = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
        summary = ""
        for sent in sentences:
            if sent in top_dict.keys():
                summary += sent
        print(summary)
    except:
        return float("NaN")


In [ ]:
import math
filename = 'articlesSet.csv'
fields = ['title','summary','content']

with open(filename,'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    
    def callback(row):
        summary = generateSummary(row['text'])
        if(type(summary)!=str):
            return
        rows = [row['title'],summary,row['text']]
        csvwriter.writerow(rows)
        
    df.apply(callback,axis=1)